# H3 - Getting Started

This notebook is a quick introduction to the H3 library. It will demonstrate the basic functionalities of the library, and provide links to H3 documentation where more details can be helpful.

**Important notes:**

- The default Python API for H3 (via `import h3` or `import h3.api.basic_str`), represents indices in their hexadecimal format as a Python strs and uses standard Python collection types like set and list. These conventions make it more natural to work with h3-py objects in Python, but come at the cost of converting back and forth between the C and Python representations.

- We will be using the *default* Python API (h3.api.basic_str) in this notebook. Therefore, whenever the term **"h3"** is used in the H3 functions in this notebook, it refers to a `string` representation of the H3 Index (which is a hexadecimal string equivalent of the actual `H3Index` 64-bit integer value).

- To learn more about h3-py APIs, click [Why are there multiple APIs?](https://uber.github.io/h3-py/api_comparison#why-are-there-multiple-apis)

In [ ]:
import h3
from h3_transformation import H3Transformation

# Version numbers for C (wrapped) libraries and the Python (wrapper).
h3.versions()

# Some helpers
def h3_info(method_name, h3) -> None:    
    info = f"""{method_name}
    - type   : {type(h3)}
    - size   : {len(h3) if isinstance(h3, (list, set)) else 1}
    - output : {h3}
    """
    
    print(info)

## Introduction

### `H3Index` - The core of H3
The H3 system assigns a unique hierarchical index (H3Index) to each cell (hexagon/pentagon).
- Each directed edge (Mode 2) and vertex (Mode 4) is assigned an index based on its origin or owner cell, respectively. 



### `H3Index` Representations

An H3Index is the 64-bit `integer` representation of an H3 index, which may be one of multiple modes to indicate the concept being indexed.
- Mode 0 is reserved and indicates an **invalid H3 index**.
- Mode 1 is an **H3 Cell** (Hexagon/Pentagon) index.
- Mode 2 is an **H3 Directed Edge** (Cell A -> Cell B) index.
- Mode 3 is planned to be a **bidirectional edge** (Cell A <-> Cell B).
- Mode 4 is an **H3 Vertex** (i.e. a single vertex of an H3 Cell).

### Key points

- By default, the keyword "h3" in H3 refers to the `string` representation the equivalent hexadecimal of the `H3Index` (a 64-bit `integer` H3 index) using lowercase letters.
- `H3Index` refers to the 64-bit `integer` representation of an H3 index.
- "H3 Index" / `H3Cell` refer to the `string` representation of the H3 index.

---

## Main concepts

We will go through the fundamental API usages of H3, please note that all the examples in this notebook are from H3 version 3.x (https://h3geo.org/docs/3.x/).

A lot of these methods will look very different in H3 version 4.x (https://h3geo.org/docs/), which is currently in beta.

Also, all the function names (e.g. `geoToH3`) & descriptions in this notebook are from the [h3geo](https://h3geo.org/docs/) documentation. Therefore, they may have different names & return inputs/outputs. However, the concepts are the same.

### Indexing functions

These function are used for finding the H3 index containing coordinates, and for finding the center and boundary of H3 indexes.

#### [geoToH3](https://h3geo.org/docs/3.x/api/indexing#geotoh3)

Indexes the location at the specified resolution, returning the index of the cell containing the location. This buckets the geographic point into the H3 grid. See the algorithm description for more information.

Returns 0 on error.

In [ ]:
latitude = 10.7780202
longitude = 106.7019901
resolution = 12  # For geodesic statistics of resolution, visit: https://h3geo.org/docs/3.x/core-library/restable

# Generate an H3 cell, which is an H3Index (string) in Mode 1 (due to our notebook is using the default H3 API).
h3_cell = h3.geo_to_h3(lat=latitude, lng=longitude, resolution=resolution)
h3_cell

#### [h3ToGeo](https://h3geo.org/docs/3.x/api/indexing#h3togeo)

Finds the center (lat, lng) of the cell in grid space. See the algorithm description for more information.

The center will drift versus the centroid of the cell on Earth due to distortion from the gnomonic projection within the icosahedron face it resides on and its distance from the center of the icosahedron face.



In [ ]:
# Returns the (lat, lng) of the center of the H3 cell
h3.h3_to_geo(h3_cell) # yes, h3_to_geo sounds pretty confusing, and H3 change this method to cell_to_latlng in v4.x

#### [h3ToGeoBoundary](https://h3geo.org/docs/3.x/api/indexing#h3togeoboundary)

Finds the boundary (vertices of the hexagon/pentagon) of the cell.

To learn more about the implementation, take a look at the documentation: [Generate the cell boundary in latitude/longitude coordinates of an H3Index cell](https://h3geo.org/docs/3.x/core-library/h3ToGeoBoundaryDesc/).

In [ ]:
boundary = h3.h3_to_geo_boundary(h3_cell, geo_json=False)
boundary_geojson = h3.h3_to_geo_boundary(
    h3_cell,
    geo_json=True,  # returns output in GeoJSON format: [lng, lat] order; first and last coordinates are identical.
)

print(boundary, boundary_geojson, sep="\n")

### Inspection functions

These functions provide metadata about an H3 index, such as its resolution or base cell, and provide utilities for converting into and out of the 64-bit representation of an H3 index.

#### [h3GetResolution](https://h3geo.org/docs/3.x/api/inspection#h3getresolution)

Returns the resolution of the index.

In [ ]:
h3.h3_get_resolution(h3_cell)

#### [h3GetBaseCell](https://h3geo.org/docs/3.x/api/inspection#h3getbasecell)

Returns the base cell number of the index.

In [ ]:
h3.h3_get_base_cell(h3_cell)

#### [stringToH3](https://h3geo.org/docs/3.x/api/inspection#stringtoh3)

Converts the string representation to `H3Index` (uint64_t) representation.

Returns 0 on error.

In [ ]:
h3_cell_uint64_t = h3.string_to_h3(h3_cell)

print(type(h3_cell_uint64_t), h3_cell_uint64_t)

#### [h3ToString](https://h3geo.org/docs/3.x/api/inspection#h3tostring)

Converts the `H3Index` (uint64_t) representation of the index to the `string` representation. str must be at least of length 17.

In [ ]:
h3_cell_str = h3.h3_to_string(h3_cell_uint64_t)

print("h3_cell_str", type(h3_cell_str), h3_cell_str, sep=' - ')    # `string` representation of the H3 Cell.
print("h3_cell (default)", type(h3_cell), h3_cell, sep=' - ')            # In the default Python API, any H3 Cell generated is already in the `string` representation.

#### [h3IsValid](https://h3geo.org/docs/3.x/api/inspection#h3isvalid)

Returns non-zero if this is a valid H3 index.

In [ ]:
h3_cell_check           = h3.h3_is_valid(h3_cell)
h3_cell_str_check       = h3.h3_is_valid(h3_cell_str)
h3_cell_uint64_t_check  = h3.h3_is_valid(h3_cell_uint64_t)

# However, it seems that `h3_is_valid` only works with `string` representation of the H3 Cell. I guess in v3.x, the term `h3` always refers to the `string` representation of the H3 Cell :-). 
h3_cell_check, h3_cell_str_check, h3_cell_uint64_t_check

#### [h3ispentagon](https://h3geo.org/docs/3.x/api/inspection#h3ispentagon)

Returns non-zero if this index represents a pentagonal cell.

*Why is there a Pentagon?*
> "The H3 grid is constructed by laying out **122 base cells** (icosahedrons) over the Earth, with ten cells per face. Some cells are contained by more than one face. Since it is not possible to tile the icosahedron with only hexagons, we chose to introduce **twelve pentagons**, one at *each of the icosahedron vertices*. These vertices were positioned using the spherical icosahedron orientation by R. Buckminster Fuller, which places **all the vertices in the water**. This helps avoid pentagons surfacing in our work." - [Uber Blog - H3, 2018](https://www.uber.com/en-VN/blog/h3/)


In [ ]:
h3.h3_is_pentagon(h3_cell)

### Traversal functions

Grid traversal allows finding cells in the vicinity of an origin cell, and determining how to traverse the grid from one cell to another.

In [ ]:
# Set distance for all traversal examples
distance_5km = 5

#### [kRing](https://h3geo.org/docs/3.x/api/traversal)

k-rings produces indices within k distance of the origin index.

k-ring 0 is defined as the origin index, k-ring 1 is defined as k-ring 0 and all neighboring indices, and so on.

Output is placed in the provided array in **no particular order**. Elements of the output array may be left zero, as can happen when crossing a pentagon.

In [ ]:
# K-ring of a H3 cell:
k_ring_5km = h3.k_ring(h=h3_cell, k=distance_5km)

h3_info("k_ring", k_ring_5km)

#### [kRingDistances / hexRangeDistances](https://h3geo.org/docs/3.x/api/traversal#kringdistances)

Alias for `hex_range_distances`:

Ordered list of the "hollow" rings around `h`, up to and including distance `K`.

In [ ]:
k_ring_distances_5km = h3.k_ring_distances(h=h3_cell, K=distance_5km)
hex_range_distances_5km = h3.hex_range_distances(h=h3_cell, K=distance_5km)

h3_info("k_ring_distances", k_ring_distances_5km)
h3_info("hex_range_distances", hex_range_distances_5km)

In [ ]:
# The output of the above methods is a list of sets - where each set holds the H3 cells of a hollow ring.
rings_5km = h3.hex_range_distances(h3_cell, distance_5km)

current_ring = 0
for ring in rings_5km:
    current_ring += 1
    print(f"Ring {current_ring} has {len(ring)} cells")

    for cell in ring:
        print(f"Cell: {cell}, Center: {h3.h3_to_geo(cell)}")

    print("-------------------")

#### [hexRing](https://h3geo.org/docs/3.x/api/traversal#hexring)

Produces the hollow hexagonal ring centered at origin with sides of length k.

Returns 0 if no pentagonal distortion was encountered.

In [ ]:
hex_ring_5km = h3.hex_ring(h=h3_cell, k=distance_5km)

h3_info("hex_ring", hex_ring_5km)

### Hierarchical functions

These functions permit moving between resolutions in the H3 grid system. The functions produce parent (coarser) or children (finer) cells.

#### [h3ToParent](https://h3geo.org/docs/3.x/api/hierarchy#h3toparent)

Returns the parent (coarser) index containing `h`.

In [ ]:
# res is resolution of the parent cell, if not specified: res = h3.h3_get_resolution(h3_cell) - 1
parent = h3.h3_to_parent(h=h3_cell, res=9)
h3_info("h3_to_parent", parent)

#### [h3ToChildren](https://h3geo.org/docs/3.x/api/hierarchy#h3tochildren)

Populates children with the indexes contained by `h` at resolution childRes. children must be an array of at least size `maxH3ToChildrenSize(h, childRes)`.

In [ ]:
# If res=None, then res = resolution(h) + 1
children = h3.h3_to_children(h=h3_cell, res=None)
h3_info("h3_to_children", children)

print("Resolution of h3_cell    :", h3.h3_get_resolution(h3_cell))
print("Resolution of children   :", h3.h3_get_resolution(list(children)[0]))

#### [h3ToCenterChild](https://h3geo.org/docs/3.x/api/hierarchy#h3tocenterchild)

Returns the center child (finer) index contained by h at resolution childRes.

In [ ]:
center_child = h3.h3_to_center_child(h=h3_cell, res=None)
h3_info("h3_to_center_child", center_child)

print("Resolution of h3_cell        :", h3.h3_get_resolution(h3_cell))
print("Resolution of center_child   :", h3.h3_get_resolution(center_child))

### Region functions

These functions convert H3 indexes to and from polygonal areas.

#### [polyfill](https://h3geo.org/docs/3.x/api/regions#polyfill)

polyfill takes a given GeoJSON-like data structure and preallocated, zeroed memory, and fills it with the hexagons that are contained by the GeoJSON-like data structure.

Containment is determined by the cells' centroids. A partitioning using the GeoJSON-like data structure, where polygons cover an area without overlap, will result in a partitioning in the H3 grid, where cells cover the same area without overlap.

In [ ]:
# h3.polyfill(polygons, res, geo_json_conformant=False)

"""
Since h3.polyfill takes a GeoJSON (Polygons) as input, the actual sample is a bit more complicated
and will be moved to a more comprehensive notebook - h3_polyfill_administrative_pipeline.ipynb

It will demonstrate an end-to-end sample pipeline which performs the following tasks:
1. Extract & process GeoJSON from PostGIS geometry (MultiPolygons GeoJSON -> Polygon GeoJSONs)
2. Generate H3 cells from the Polygon GeoJSONs & combine into a single set of H3 cells
3. Save the H3 cells to PostGresSQL
"""

## H3 to GeoJSON

H3Transformation is a custom class that I made to convert H3 cells to GeoJSON. Currently, it supports 2 methods:
1. `cell_to_geojson` - Converts a single (`str`) H3 cell to a GeoJSON (Polygon/FeatureCollection).
2. `cells_to_geojson` - Converts a (`set<str>`) of H3 cells to a GeoJSON (Polygon/FeatureCollection).

You can copy the print result of `geojson` to [geojson.io](https://geojson.io/) to visualize the result.

In [ ]:
geojson = H3Transformation.cells_to_geojson(hex_ring_5km)
print(geojson)